# Mise en forme des données

Ici on prend les données original et on les mets dans currated

In [1]:
import pandas as pd
import numpy as np

In [28]:
# Répertoire des données d'origine.

dataOriginalPath = 'Data/Original/'
dataTrainX = dataOriginalPath +'clickbait17-train-170630/instances.jsonl'
dataTrainY = dataOriginalPath +'clickbait17-train-170630/truth.jsonl'
dataTestX = dataOriginalPath +'clickbait17-test-170720/instances.jsonl'
dataTestY = dataOriginalPath +'clickbait17-test-170720/truth.jsonl'

# Le dataset va être sauvegardé là (on conserve tous les sites à cette étape du calcul) :
dataCuratedPath = 'Data/Curated/'
LCfraction = 'Fraction'

In [24]:
dataTrainX

'Data/Original/clickbait17-train-170630/instances.jsonl'

In [30]:
pdoTrainX=pd.read_json(dataTrainX, lines = True)

In [31]:
pdoTrainY=pd.read_json(dataTrainY, lines = True)

In [32]:
pdoTrainY

,truthJudgments,truthMean,id,truthClass,truthMedian,truthMode
0,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,858464162594172928,clickbait,1.000000,1.000000
1,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,858462320779026432,no-clickbait,0.000000,0.000000
2,"[0.33333333330000003, 0.6666666666000001, 1.0,...",0.400000,858460992073863168,no-clickbait,0.333333,0.000000
3,"[0.0, 0.6666666666000001, 0.0, 0.3333333333000...",0.266667,858459539296980992,no-clickbait,0.333333,0.333333
4,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,858455355948384256,no-clickbait,0.000000,0.000000
...,...,...,...,...,...,...
19533,"[0.0, 0.6666666666000001, 0.0, 0.0, 0.0]",0.133333,804126501117435904,no-clickbait,0.000000,0.000000
19534,"[0.0, 0.0, 0.0, 0.33333333330000003, 0.0]",0.066667,804123103995580416,no-clickbait,0.000000,0.000000
19535,"[0.6666666666000001, 0.6666666666000001, 0.0, ...",0.333333,804121272967983104,no-clickbait,0.333333,0.000000
19536,"[1.0, 0.0, 0.6666666666000001, 1.0, 1.0]",0.733333,804119512010424320,clickbait,1.000000,1.000000


In [33]:
# Il faut convertir le résultat sous forme de dataset
curated = ds.Dataset.from_pandas(pdo2) #, preserve_index=False)

NameError: name 'ds' is not defined

In [34]:
curated.train_test_split(0.2)

NameError: name 'curated' is not defined

In [35]:
curated.info.description = "QuebecTop clicmeter"
curated.info.version = "0.3.0"
curated.info.supervised_keys = [LCfraction]
curated.save_to_disk(dataCuratedPath)

NameError: name 'curated' is not defined